In [1]:
#source: https://wiki.fysik.dtu.dk/gpaw/tutorialsexercises/electronic/band_gap/band_gap.html
from ase.build import bulk
from gpaw import GPAW, PW, FermiDirac

# Ground state calculation

def get_band_gap(atoms=None, cutoff=500,kpts=[7,7,7]):
    calc = GPAW(mode=PW(cutoff),
                xc='GLLBSC',
                kpts=kpts,  # Choose and converge carefully!
                occupations=FermiDirac(0.01),
                txt='gs.out')
    atoms.calc = calc
    atoms.get_potential_energy()
    # Calculate the discontinuity potential and the discontinuity
    homo, lumo = calc.get_homo_lumo()
    response = calc.hamiltonian.xc.response
    dxc_pot = response.calculate_discontinuity_potential(homo, lumo)
    KS_gap, dxc = response.calculate_discontinuity(dxc_pot)

    # Fundamental band gap = Kohn-Sham band gap + derivative discontinuity
    QP_gap = KS_gap + dxc

    print(f'Kohn-Sham band gap:         {KS_gap:.2f} eV')
    print(f'Discontinuity from GLLB-sc: {dxc:.2f} eV')
    print(f'Fundamental band gap:       {QP_gap:.2f} eV')
    return QP_gap,KS_gap,dxc

In [2]:
# atoms = bulk('Si', 'diamond', 5.431)
# QP_gap,KS_gap,dxc = get_band_gap(atoms=atoms)

In [3]:
import pandas as pd
df=pd.read_csv('https://github.com/usnistgov/jarvis_leaderboard/raw/main/jarvis_leaderboard/contributions/vasp_tbmbj/ES-SinglePropertyPrediction-bandgap-dft_3d-test-mae.csv.zip')

In [4]:
df

,id,target,prediction
0,JVASP-1002,1.17,1.277
1,JVASP-104,3.40,2.474
2,JVASP-113,5.50,4.213
3,JVASP-1130,14.20,11.219
4,JVASP-1145,8.50,8.409
5,JVASP-116,7.83,6.805
6,JVASP-1174,1.52,1.321
7,JVASP-1180,0.72,0.758
8,JVASP-1183,1.42,1.387
9,JVASP-1189,0.24,0.134


In [ ]:
import pandas as pd
from gpaw import GPAW, PW, FermiDirac
from jarvis.db.figshare import get_jid_data
from jarvis.core.atoms import Atoms
from jarvis.core.kpoints import Kpoints3D
import time

for i,ii in df.iterrows():
    try:
        jid = ii['id']
        print('jid',jid)
        dat=get_jid_data(dataset='dft_3d',jid=jid)
        atoms=Atoms.from_dict(dat['atoms']).ase_converter()
        kp = Kpoints3D().automatic_length_mesh(

            lattice_mat=dat['atoms']['lattice_mat'],
            length=dat["kpoint_length_unit"],
        )
        kpts = kp._kpoints[0]
        t1=time.time()
        QP_gap,KS_gap,dxc = get_band_gap(atoms=atoms,kpts=kpts)
        t2=time.time()
        name=jid.replace('-','_')+'_'+atoms.composition.reduced_formula
        fname='ES-SinglePropertyPrediction-bandgap_'+name+'-dft_3d-test-mae.csv'
        f=open(fname,'w')
        line='id,prediction\n'
        f.write(line)
        print('jid,QP_gap,KS_gap,dxc',jid,QP_gap,KS_gap,dxc,t2-t1)
        line=jid+','+str(QP_gap)+'\n'
        f.write(line)
        f.write(line)
        f.close()
    except:
       print('Failed for jid', jid)
       pass

       

jid JVASP-1002
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
Kohn-Sham band gap:         0.79 eV
Discontinuity from GLLB-sc: 0.37 eV
Fundamental band gap:       1.17 eV
Failed for jid JVASP-1002
jid JVASP-104
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
Failed for jid JVASP-104
jid JVASP-113
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.
Kohn-Sham band gap:         5.21 eV
Discontinuity from GLLB-sc: 1.57 eV
Fundamental band gap:       6.79 eV
Failed for jid JVASP-113
jid JVASP-1130
Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524